# Tutorial 2 – Liquid Chromatography

Liquid chromatography (LC) separates peptides in time before they enter the mass spectrometer. By separating them first by LC, the mass spectrometer only needs to co-analyze smaller groups of peptides, increasing the overall accuracy and reliability of the results. In bottom-up proteomics we typically use **reverse-phase LC**, where the stationary phase is hydrophobic and the mobile phase gets gradually more organic. Hydrophobic peptides stick longer and elute later, while hydrophilic peptides elute early.

In this tutorial we recreate the LC step computationally: we start from the digested peptides, predict their retention times using a hydrophobicity model, and visualize how well the chromatographic gradient separates them.

## Objectives
- Learn why separating peptides via LC before MS improves detection accuracy.
- Use a retention time prediction model to estimate LC retention times based on peptide hydrophobicity.
- Plot histograms of predicted retention times to analyze how peptides are separated.
- Practice using external scientific libraries and Matplotlib for visualization.

---

## 1. Load the digested peptides
We start where Tutorial 1 ended: by reusing the pickled `digested_peptides_map` generated after protein digestion. Each entry maps a protein ID to the list of peptides produced by your chosen protease.

> 🚩 **Fill in the triple dots [...]**

In [ ]:
import pickle

pickle_path = ...  # e.g., '../results/digested_peptides_map.pkl'
with open(pickle_path, 'rb') as handle:
    digested_peptides_map = ...

len(digested_peptides_map)

### 1.1 Pool peptides across proteins
For LC we only need the peptide list, not the protein grouping. Flatten the dictionary values into one list and report how many peptides you have in total.

> 🚩 **Fill in the triple dots [...]**

In [ ]:
pooled_peptides = ...  
print(f'Total peptides: {len(...)}')

---

## 2. Predict LC retention times
Modern LC-MS pipelines often rely on machine learning models that map peptide sequences to retention times by encoding amino-acid properties (hydrophobicity, charge, sequence motifs). Tools like DeepLC, Prosit, and ELUDE have become staples in proteomics workflows. In this tutorial we use the `pyteomics.achrom` model, which approximates reverse-phase LC behavior using well-characterized hydrophobicity coefficients.

In [ ]:
# Install pyteomics if needed
!pip install -q pyteomics

from pyteomics import achrom

The variable ```achrom.RCs_guo_ph7_0``` specifies the use of a predefined set of retention coefficients based on experimental data from Guo et al., acquired at a pH of 7.0. These coefficients are suitable to estimate RTs of peptides under similar conditions. As a demonstration, below the relative retention time is computed for the example sequence ‘PEPTIDE’.

**Important:** the `achrom` model returns *relative, unitless* retention times. They describe the ordering of peptides along the gradient, not absolute minutes. We still label axes as minutes to mimic real LC outputs, but remember these are pseudo minutes for visualization only. For the rest of this tutorial and downstream processes, we will continue to use the relative retention time for convenience.

In [ ]:
relative_RT = achrom.calculate_RT('PEPTIDE', achrom.RCs_guo_ph7_0)
print("The relative RT of the amino acid sequence PEPTIDE is: ", relative_RT)

The relative RT of the amino acid sequence PEPTIDE is:  7.800000000000001


## ⭐ Essential function: `predict_lc_retention_times`
Now let's make a function which takes a list of peptide sequences and returns a dictionary mapping each peptide to its predicted retention time (rounded to two decimals).

> 🚩 **Fill in the triple dots [...]**

In [ ]:
def predict_lc_retention_times(peptides):
    """
    Add a short description here.

    Parameters
    ----------

    Returns
    -------
    """

    ...

Now apply the function to our `pooled_peptides` list.

In [ ]:
rt_map = ...

rt_map

What are the minimum and maximum RTs?

In [ ]:
min_rt = ...
max_rt = ...

print(f'Min: {min_rt}')
print(f'Max: {max_rt}')

Do you observe any unexpected results? If so, what do you think is explanation?
<details>
<summary> Solution </summary>
You might be wondering how there can be negative RTs?

This is because the prediction tool only provides relative RT values that need to be calibrated based on actually observed RTs of peptides in a given chromatographic setup. This will translate the relative and unitless RTs into actual RTs in minutes.
</details>

### ✅ Unit test: `predict_lc_retention_times`
Create a dummy peptide list, run your function, and assert the output matches the expected dictionary (rounded to two decimals).

> 🚩 **Fill in the triple dots [...]**

In [ ]:
def test_predict_lc_retention_times():
    peptides = ...
    expected = ...

    actual = predict_lc_retention_times(peptides)
    assert actual == expected

test_predict_lc_retention_times()

---

## 3. Visualize retention time distributions
Plotting retention times as histograms shows how many peptides elute at different parts of the LC gradient. A smooth, broad distribution indicates that the chromatographic method separates peptides across the whole run, reducing MS undersampling.

We will use Matplotlib histograms where the x-axis is labeled as retention time (minutes) so the plot looks like a real chromatogram, even though the predicted values are unitless and relative. The y-axis approximates intensity by counting how many peptides fall into each bin.

## ⭐ Essential function: `plot_retention_time`
Wrap the histogram plotting into a reusable function that accepts a list (or dict values) of retention times plus a `resolution` parameter controlling the number of histogram bins.

> 🚩 **Fill in the triple dots [...]**

In [ ]:
import matplotlib.pyplot as plt

def plot_retention_time(retention_times, resolution=30):
    """
    Add a short description here.

    Parameters
    ----------

    Returns
    -------

    """
    ...

### Exercise: compare resolutions
Use your plotting function to visualize the same retention-time data twice—once with `resolution=30` bins and once with `resolution=60`. Describe how the bin size changes the appearance and interpretability of the chromatogram-like histogram.

> 🚩 **Fill in the triple dots [...]**

In [ ]:
...

**Questions:**

- What do you see? How do peptides distribute across different bins?
- What happens if you increase the number of bins?
- Investigating different numbers of bins roughly mimics looking at different chromatographic resolutions or gradient lengths. Can you imagine how increasing the resolution impacts MS analysis?

---

## 4. Select a retention-time window
After predicting retention times, LC-MS/MS experiments typically focus on a specific time window for MS1/MS2 acquisition. Implementing a selection function lets us mimic that filtering step computationally.

The function will accept the retention-time dictionary and lower/upper bounds (in minutes) and return only the peptides that fall inside that window.

> 🚩 **Fill in the triple dots [...]**

In [ ]:
def select_retention_time_window(peptide_rt_map, lower_ret_time, upper_ret_time):
    """
    Add a short description here.

    Parameters
    ----------

    Returns
    -------

    """
    ...

### 4.1 Apply the window filter
Use your function to select peptides that elute between 0 and 20 minutes. Store them in `select_peps` — this subset will feed into the MS1 simulation in the next tutorial.

> 🚩 **Fill in the triple dots [...]**

In [ ]:
select_peps = select_retention_time_window(
    peptide_rt_map=...,
    lower_ret_time=0,
    upper_ret_time=20,
)

len(select_peps)

### ✅ Unit test: `select_retention_time_window`
Create a small retention-time dictionary (2–3 peptides) and assert that the helper returns exactly those peptides that fall within a chosen window. Also test that peptides outside the range are excluded.

> 🚩 **Fill in the triple dots [...]**

In [ ]:
def test_select_retention_time_window():
    peptide_rt_map = {...}
    selected = select_retention_time_window(peptide_rt_map, lower_ret_time=..., upper_ret_time=...)

    assert selected == ...

test_select_retention_time_window()

---

## 5. Save the selected peptides
Persist the `select_peps` list so you can reuse it in the MS tutorial. Saving it as a pickle file avoids recomputing the retention times if you tweak parameters later.

### 5.1 Write to disk

> 🚩 **Fill in the triple dots [...]**

In [ ]:
import pickle
output_path = '../results/select_peptides.pkl'
with open(output_path, 'wb') as file:
    pickle.dump(..., file)